# Imports

In [1]:
import numpy as np
import pandas as pd
from numba import njit
from itertools import product
from ortools.linear_solver import pywraplp
from multiprocessing import Process

# cvs2mst

In [108]:
data = pd.read_csv('../data/family_data.csv')
n_neople = data.n_people
data = data.drop(['family_id', 'n_people'], axis=1)

In [109]:
N_DAYS = 100
N_FAMILIES = 5000
MAX_OCCUPANCY = 300
MIN_OCCUPANCY = 125

FAMILY_SIZE = n_neople.values
DESIRED     = data.values[:, :-1] - 1

In [112]:
kernel_csv = pd.read_csv('../submissions/best.csv')

In [138]:
kernel_csv.assigned_day.max(), kernel_csv.assigned_day.min()

(100, 1)

In [147]:
kernel_occs = []
for day in range(1,N_DAYS+1):
    tmp_id = kernel_csv[kernel_csv.assigned_day==day].family_id
    kernel_occs.append(n_neople[tmp_id].sum())

In [159]:
kernel_occs[47], kernel_occs[48]

(219, 188)

In [160]:
f = open('../practice/kernel_init.mst', 'w')
f.write('# MIP start\n')

for index in range(N_FAMILIES):
    choice_days = data.iloc[index, :]
    for choice in choice_days:
        f.write('x[%i,%i] ' %(index, choice))
        if kernel_csv.iloc[index, :][1] == choice:
            f.write('1\n')
        else:
            f.write('0\n')
        
for day in range(N_DAYS):
    for occ_now in range(MIN_OCCUPANCY,MAX_OCCUPANCY+1):
        for occ_past in range(MIN_OCCUPANCY,MAX_OCCUPANCY+1):
            f.write('N[%i,%i,%i] ' %(day, occ_now, occ_past))
            if day == N_DAYS-1:
                f.write('0\n')
            elif (occ_now==kernel_occs[day]) and (occ_past==kernel_occs[day+1]):
                f.write('1\n')
            else:
                f.write('0\n')
            
f.close()